In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
# import os

# print(os.getcwd())
# os.chdir('')
# print(os.getcwd())


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import os
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, CuDNNLSTM
import keras.layers as layers
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping
import splitSequence as sq
from math import ceil
from sklearn.preprocessing import MinMaxScaler
import lstmModels as mdl
import transformData as tr
import json
#!pip install ta
import ta
import tensorflow as tf
# from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [57]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1916534211855674956
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6657650197
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9327665238469921408
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'experimental_list_devices'

In [41]:
os.chdir('../inputData/')

In [38]:
params = {
"lstm_layer1":[100, 150, 200],
"dropout1":[0.2, 0.5],
"lstm_layer2":[50, 100, 150],  
"dropout2":[0.2, 0.5]
}
['best_epoch','val_loss'] + list(params.keys())

['best_epoch',
 'val_loss',
 'lstm_layer1',
 'dropout1',
 'lstm_layer2',
 'dropout2']

In [49]:
resultsframe = pd.DataFrame(columns=['best_epoch','val_loss', 'training_loss'] + list(params.keys()))
resultsframe

,best_epoch,val_loss,training_loss,lstm_layer1,dropout1,lstm_layer2,dropout2


In [53]:
files[0]

'aep.us.txt'

In [60]:
files = os.listdir('./stocksLong')
path = files[0]
for topLayer in params['lstm_layer1']:
    for dropout1 in params['dropout1']:
        for botLayer in params['lstm_layer2']:
            for dropout2 in params['dropout2']:
                data = pd.read_csv(f'./stocksLong/{path}', parse_dates=True).drop(columns=['OpenInt'])
                # We remove the last part of the data, because the market doubled in that timeframe
                data = data.iloc[:int(len(data) * 0.8)]
                name = path.split('.')[0]
                folder = 'savedModels/run14-testingGPU-longstocks-150-75cells'
                run_name = f'run14-01-longstocks-gridsearch-40epochs-params-{topLayer}-LSTM{dropout1}-drop1{botLayer}-LSTM{dropout2}-drop2'
                length = 50 
                features = 7
                epochs = 40
                input_shape = (length, features)
                Xcol=['Open', 'High', 'Low', 'Close', 'Volume', 'trend_macd', 'others_dr']
                ycol='Close'
                data = ta.add_all_ta_features(data, open = 'Open', high = 'High', low= 'Low', close='Close', volume='Volume', fillna=True).set_index('Date')
                data = data[Xcol]
                split_index = ceil(0.8 * len(data))

                # Scale the data on the min maximum of the training set
                dataScaled = tr.minMaxScaleOnTrain(data)
                genTrain, genTest = tr.getTrainTestGenerator(dataScaled, Xcol=Xcol, ycol=ycol, length=length, batch_size=128, stride=10)

                print(f'testing model with {topLayer}-LSTM{dropout1}-drop1{botLayer}-LSTM{dropout2}-drop2')
                
                # Create the model, train it and save it.
                model = Sequential()
                model.add(LSTM(topLayer, input_shape=input_shape, return_sequences=True))
                model.add(Dropout(dropout1))
                model.add(LSTM(botLayer, return_sequences=False))
                model.add(Dropout(dropout2))
                model.add(Dense(1, activation="sigmoid", kernel_initializer="uniform"))
                model.compile(optimizer='adam',loss='mean_squared_error')

                history = model.fit_generator(genTrain, epochs=epochs, validation_data=genTest, use_multiprocessing=True)
                model.save(f'../{folder}/{name}-{run_name}.json')

                # Save metrics as csv
                histDF = pd.DataFrame.from_dict(history.history)
                histDF.to_csv(f'../{folder}/history-{name}-{run_name}.csv')
                minLoss, minLossEpoch = histDF.val_loss.min(), histDF.val_loss.idxmin() + 1
                resultsframe = resultsframe.append({
                    'best_epoch': minLossEpoch,
                    'val_loss': minLoss,
                    'lstm_layer1':topLayer,
                     'dropout1': dropout1,
                     'lstm_layer2': botLayer,
                     'dropout2': dropout2
                }, ignore_index=True)
                resultsframe.to_csv(f'../{folder}/bestRunStatistics-{name}.csv')


                # Predict xtest and ytest.
                genAll = TimeseriesGenerator(data=np.array(dataScaled[Xcol]), targets=np.array(dataScaled[ycol]), length=length, batch_size=1)
                ypred = model.predict_generator(genAll)
                ytest = dataScaled[ycol].iloc[length:]
                realvsPred = pd.concat((ytest.reset_index(), pd.DataFrame(ypred, columns=[f'pred{ycol}'])), axis=1)
                realvsPred['Date'] = pd.to_datetime(realvsPred['Date'])
                realvsPred.set_index('Date')
                realvsPred.to_csv(f'../{folder}/{name}-{run_name}.csv', index=False)

C:\Users\Eduardo\Anaconda3\lib\site-packages\ta\trend.py:543: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
C:\Users\Eduardo\Anaconda3\lib\site-packages\ta\trend.py:547: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


testing model with 100-LSTM0.2-drop150-LSTM0.2-drop2
Epoch 1/40
6/6 [==============================] - 4s 639ms/step - loss: 0.1308 - val_loss: 0.3291
Epoch 2/40
6/6 [==============================] - 3s 525ms/step - loss: 0.1155 - val_loss: 0.3466
Epoch 3/40
6/6 [==============================] - 3s 510ms/step - loss: 0.0987 - val_loss: 0.5244
Epoch 4/40
6/6 [==============================] - 3s 577ms/step - loss: 0.0751 - val_loss: 0.5767
Epoch 5/40
6/6 [==============================] - 4s 626ms/step - loss: 0.0708 - val_loss: 0.5748
Epoch 6/40
6/6 [==============================] - 3s 568ms/step - loss: 0.0687 - val_loss: 0.5653
Epoch 7/40
6/6 [==============================] - 3s 568ms/step - loss: 0.0683 - val_loss: 0.5471
Epoch 8/40
6/6 [==============================] - 3s 582ms/step - loss: 0.0576 - val_loss: 0.3305
Epoch 9/40
6/6 [==============================] - 3s 537ms/step - loss: 0.0378 - val_loss: 0.2402
Epoch 10/40
6/6 [==============================] - 3s 572ms/step 

C:\Users\Eduardo\Anaconda3\lib\site-packages\ta\trend.py:543: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
C:\Users\Eduardo\Anaconda3\lib\site-packages\ta\trend.py:547: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


testing model with 100-LSTM0.2-drop150-LSTM0.5-drop2
Epoch 1/40
6/6 [==============================] - 5s 794ms/step - loss: 0.1296 - val_loss: 0.3236
Epoch 2/40
6/6 [==============================] - 4s 663ms/step - loss: 0.1179 - val_loss: 0.3693
Epoch 3/40
6/6 [==============================] - 4s 586ms/step - loss: 0.0870 - val_loss: 0.3400
Epoch 4/40
6/6 [==============================] - 4s 603ms/step - loss: 0.0806 - val_loss: 0.2861
Epoch 5/40
6/6 [==============================] - 4s 598ms/step - loss: 0.0438 - val_loss: 0.2463
Epoch 6/40
6/6 [==============================] - 4s 605ms/step - loss: 0.0305 - val_loss: 0.2348
Epoch 7/40
6/6 [==============================] - 4s 610ms/step - loss: 0.0269 - val_loss: 0.2256
Epoch 8/40
6/6 [==============================] - 4s 622ms/step - loss: 0.0320 - val_loss: 0.2198
Epoch 9/40
6/6 [==============================] - 4s 663ms/step - loss: 0.0174 - val_loss: 0.2197
Epoch 10/40
6/6 [==============================] - 5s 872ms/step 

C:\Users\Eduardo\Anaconda3\lib\site-packages\ta\trend.py:543: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
C:\Users\Eduardo\Anaconda3\lib\site-packages\ta\trend.py:547: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


...

testing model with 200-LSTM0.5-drop1100-LSTM0.5-drop2
Epoch 1/40
6/6 [==============================] - 3s 582ms/step - loss: 0.1334 - val_loss: 0.4303
Epoch 2/40
6/6 [==============================] - 3s 456ms/step - loss: 0.0939 - val_loss: 0.2767
Epoch 3/40
6/6 [==============================] - 3s 487ms/step - loss: 0.0450 - val_loss: 0.1892
Epoch 4/40
6/6 [==============================] - 3s 456ms/step - loss: 0.0205 - val_loss: 0.1707
Epoch 5/40
6/6 [==============================] - 3s 477ms/step - loss: 0.0164 - val_loss: 0.1638
Epoch 6/40
6/6 [==============================] - 3s 464ms/step - loss: 0.0160 - val_loss: 0.1754
Epoch 7/40
6/6 [==============================] - 3s 476ms/step - loss: 0.0155 - val_loss: 0.1816
Epoch 8/40
6/6 [==============================] - 3s 461ms/step - loss: 0.0149 - val_loss: 0.1893
Epoch 9/40
6/6 [==============================] - 3s 472ms/step - loss: 0.0175 - val_loss: 0.1943
Epoch 10/40
6/6 [==============================] - 3s 460ms/step

C:\Users\Eduardo\Anaconda3\lib\site-packages\ta\trend.py:543: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
C:\Users\Eduardo\Anaconda3\lib\site-packages\ta\trend.py:547: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


testing model with 200-LSTM0.5-drop1150-LSTM0.2-drop2
Epoch 1/40
6/6 [==============================] - 3s 561ms/step - loss: 0.1191 - val_loss: 0.3769
Epoch 2/40
6/6 [==============================] - 3s 473ms/step - loss: 0.0589 - val_loss: 0.2174
Epoch 3/40
6/6 [==============================] - 3s 524ms/step - loss: 0.0139 - val_loss: 0.1470
Epoch 4/40
6/6 [==============================] - 4s 625ms/step - loss: 0.0426 - val_loss: 0.1422
Epoch 5/40
6/6 [==============================] - 3s 525ms/step - loss: 0.0295 - val_loss: 0.2187
Epoch 6/40
6/6 [==============================] - 3s 516ms/step - loss: 0.0093 - val_loss: 0.2466
Epoch 7/40
6/6 [==============================] - 3s 471ms/step - loss: 0.0100 - val_loss: 0.2315
Epoch 8/40
6/6 [==============================] - 3s 542ms/step - loss: 0.0091 - val_loss: 0.2074
Epoch 9/40
6/6 [==============================] - 4s 619ms/step - loss: 0.0098 - val_loss: 0.1843
Epoch 10/40
6/6 [==============================] - 3s 543ms/step

C:\Users\Eduardo\Anaconda3\lib\site-packages\ta\trend.py:543: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
C:\Users\Eduardo\Anaconda3\lib\site-packages\ta\trend.py:547: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


testing model with 200-LSTM0.5-drop1150-LSTM0.5-drop2
Epoch 1/40
6/6 [==============================] - 3s 570ms/step - loss: 0.1328 - val_loss: 0.6002
Epoch 2/40
6/6 [==============================] - 3s 503ms/step - loss: 0.0729 - val_loss: 0.5430
Epoch 3/40
6/6 [==============================] - 3s 525ms/step - loss: 0.0590 - val_loss: 0.2492
Epoch 4/40
6/6 [==============================] - 3s 471ms/step - loss: 0.0144 - val_loss: 0.1565
Epoch 5/40
6/6 [==============================] - 3s 496ms/step - loss: 0.0175 - val_loss: 0.1273
Epoch 6/40
6/6 [==============================] - 3s 505ms/step - loss: 0.0169 - val_loss: 0.1403
Epoch 7/40
6/6 [==============================] - 3s 518ms/step - loss: 0.0212 - val_loss: 0.1816
Epoch 8/40
6/6 [==============================] - 3s 513ms/step - loss: 0.0152 - val_loss: 0.2062
Epoch 9/40
6/6 [==============================] - 3s 475ms/step - loss: 0.0140 - val_loss: 0.2239
Epoch 10/40
6/6 [==============================] - 3s 490ms/step

In [47]:
resultsframe = resultsframe.append({   'best_epoch': 1,
                        'val_loss': 2,
                        'lstm_layer1':3,
                         'dropout1': 4,
                         'lstm_layer2': 5,
                         'dropout2': 6
                    }, ignore_index=True)
resultsframe

,best_epoch,val_loss,training_loss,lstm_layer1,dropout1,lstm_layer2,dropout2,val_los
0,1.0,NaN,NaN,3.0,4.0,5.0,6.0,2.0


In [20]:
print()

0.35852763056755066
0


In [ ]:
print(ytest.shape, ytest.shape)

(7472,) (7472,)


In [ ]:
display(ypredDF)
display(ytestDF)

,yPredict
0,0.237037
1,0.236919
2,0.236923
3,0.236941
4,0.236950
...,...
7467,0.764637
7468,0.764587
7469,0.764534
7470,0.764617


,yReal
0,0.010091
1,0.010391
2,0.010391
3,0.010684
4,0.010684
...,...
7467,1.444441
7468,1.437700
7469,1.477807
7470,1.476796


In [ ]:
#pd.DataFrame(ypred[:,0], columns=['yPredict']).merge(pd.DataFrame(ytest, columns=['yReal'])
testPred = pd.concat((ytest.reset_index(), pd.DataFrame(ypred, columns=['predClose'])), axis=1)
testPred['Date'] = pd.to_datetime(testPred['Date'])
testPred.set_index('Date')


,Close,predClose
Date,,
1988-03-11,0.010091,0.237037
1988-03-14,0.010391,0.236919
1988-03-15,0.010391,0.236923
1988-03-16,0.010684,0.236941
1988-03-17,0.010684,0.236950
...,...,...
2017-11-06,1.444441,0.764637
2017-11-07,1.437700,0.764587
2017-11-08,1.477807,0.764534
